In [25]:
class Point:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def manhatten_distance(self, other):
        return abs(self.x - other.x) + abs(self.y - other.y)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __repr__(self):
        return f'{self.__class__.__name__}({self.x}, {self.y})'
    
    

class Segment:
    
    def __init__(self, start, end, points):
        self.start = start
        self.end = end
        self.points = points
        
    def signal_delay(self, destination):
        delay = 0
        for point in self.points:
            if point == destination:
                break
            delay += 1
        return delay
    
    @classmethod
    def from_instruction(cls, start, instruction):        
        direction, length = instruction[0], int(instruction[1:])
        
        if direction == 'R':
            points = tuple(Point(start.x+i, start.y) for i in range(length+1))
        
        if direction == 'L':
            points = tuple(Point(start.x-i, start.y) for i in range(length+1))
        
        if direction == 'U':
            points = tuple(Point(start.x, start.y+i) for i in range(length+1))
        
        if direction == 'D':
            points = tuple(Point(start.x, start.y-i) for i in range(length+1))
        
        return cls(start=points[0], end=points[-1], points=points)

    

class Wire:
    
    def __init__(self, origin, segments):
        self.origin = origin
        self.segments = segments
        self.points = set(point for segment in segments for point in segment.points)
        
    def intersection(self, other, ignore_origin=True):
        shared_points = self.points.intersection(other.points)
        
        if ignore_origin and self.origin in shared_points:
            shared_points.remove(self.origin)
        
        return shared_points
    
    def signal_delay(self, destination):
        delay = 0
        for segment in self.segments:
            delay += segment.signal_delay(destination)
            if destination in segment.points:
                break
                
            # the end point of one segment is also the start point
            # of the next segment, so we remove the end point of
            # each segment that went through a full iteration
            delay -= 1
        return delay
    
    @classmethod
    def from_instructions(cls, instructions, origin=Point(0,0)):
        start = origin
        segments = []
        
        for instruction in instructions:
            segment = Segment.from_instruction(start, instruction)
            segments.append(segment)
            start = segment.end
            
        return cls(origin, segments)
    

In [24]:
test_instructions_1 = ["R75", "D30", "R83", "U83", "L12", "D49", "R7", "U7", "L72"]
test_instructions_2 = ["U62", "R66", "U55", "R34", "D71", "R55", "D58", "R83"]

test_wire_1 = Wire.from_instructions(test_instructions_1, origin)
test_wire_2 = Wire.from_instructions(test_instructions_2, origin)

test_intersections = test_wire_1.intersection(test_wire_2, ignore_origin=True)
test_signal_delays = [
    test_wire_1.signal_delay(intersection) + test_wire_2.signal_delay(intersection)
    for intersection in test_intersections
]

print(sorted(test_signal_delays))

[610, 624]


In [26]:
from pathlib import Path

instruction_1, instruction_2 = Path('./input.txt').read_text().split()
instruction_1 = instruction_1.split(',')
instruction_2 = instruction_2.split(',')

origin = Point(0, 0)

wire_1 = Wire.from_instructions(instruction_1, origin)
wire_2 = Wire.from_instructions(instruction_2, origin)

In [27]:
intersections = wire_1.intersection(wire_2, ignore_origin=True)
signal_delays = [
    wire_1.signal_delay(intersection) + wire_2.signal_delay(intersection)
    for intersection in intersections
]

print(sorted(signal_delays))

[27890, 28092, 29124, 30522, 32586, 43490, 44268]
